ETL <br>
En esta notebook tendrán todo el proceso de extracción, transformación y carga(ETL) del dataframe suser_reviews.json.

In [1]:
import pandas as pd
import numpy as np
import json 
import ast

Extracción de la data proveniente del archivo json

Se crea una variable para almacenar la ruta del archivo users_items.json para que su posterior uso ayude a tener  un codigo más legible.

In [2]:
items = "C:\\Users\\saulz\\proyecto_individual\\Dataset\\users_items.json\\australian_users_items.json"

-Comienza la extracción de los datos para cada dataframe respectivamente. <br>
-Se crea una lista vacia para almacenar la información del archivo. <br>
-La lista se transforma en dataframes usando la libreria Pandas. <br>

In [3]:
lista2= []
with open (items, encoding='utf-8') as file_items:
    for line in file_items.readlines():
        lista2.append(ast.literal_eval(line))

itemsdf = pd.DataFrame(lista2)

Transformación de los datos. <br>
-Se realizan las siguientes transformaciones con la finalidad de tener un df menos pesado y unicamente con los datos necesarios. <br>
-Se crea una mascara para hacer más facil la corroboración de datos. <br>
-Se verifican si existen datos anidados en el df. <br>
-Se eliman las siguientes columnas: 'user_url', 'funny', 'posted', 'last_edited', 'reviews', 'helpful'. <br>

In [4]:
mascara = itemsdf['user_id'] == itemsdf ['user_url']
filas_mascara = itemsdf[mascara]
filas_mascara.head(5)

,user_id,items_count,steam_id,user_url,items


Verificación de la existencia de datos anidados en el dataframe

In [5]:
if any(isinstance(valor,(list,dict,tuple)) for valor in itemsdf['items']):
    print('La columna tiene datos anidados')
else:
    print('La columna no tiene datos anidados')

La columna tiene datos anidados


In [6]:
itemsdf_expandido = itemsdf.explode('items')
itemsdf_desanidado = pd.json_normalize(itemsdf_expandido['items'])

itemsdf_expandido = itemsdf_expandido.reset_index(drop=True)
itemsdf_desanidado = itemsdf_desanidado.reset_index(drop=True)

itemsdf_final = pd.concat([itemsdf_expandido, itemsdf_desanidado], axis=1)
itemsdf_final.head(3)

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30,Day of Defeat,7.0,0.0


Se realiza un cheque de la información del dataframe una vez que ya se desanido 

In [7]:
itemsdf_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   items             object 
 5   item_id           object 
 6   item_name         object 
 7   playtime_forever  float64
 8   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 355.0+ MB


Se procede a la eliminación de las columnas que no serán relevantes para el proceso.

In [8]:
#Se eliminan las columnas de itemsdf
itemsdf_final.drop(['items', 'playtime_2weeks', 'user_url','item_name'], axis=1, inplace=True)
itemsdf_final

,user_id,items_count,steam_id,item_id,playtime_forever
0,76561197970982479,277,76561197970982479,10,6.0
1,76561197970982479,277,76561197970982479,20,0.0
2,76561197970982479,277,76561197970982479,30,7.0
3,76561197970982479,277,76561197970982479,40,0.0
4,76561197970982479,277,76561197970982479,50,0.0
...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,373330,0.0
5170011,76561198329548331,7,76561198329548331,388490,3.0
5170012,76561198329548331,7,76561198329548331,521570,4.0
5170013,76561198329548331,7,76561198329548331,519140,3.0


Se almacena el dataframe en un archivo csv para usarlo en la etapa de EDA

In [9]:
ruta= "C:/Users/saulz/proyecto_individual/Dataset csv"
itemsdf_final.to_csv('itemsdf_final.csv', index=False)